In [1]:
import pandas as pd
import numpy as numpy

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [128]:
! pip install kaggle

In [28]:
path = r"training.1600000.processed.noemoticon\training.1600000.processed.noemoticon.csv"


In [29]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rishabh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing

In [33]:
twitter_data = pd.read_csv(path, encoding='iso-8859-1')

In [34]:
twitter_data.head()

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [43]:
twitter_data.shape

(1048573, 6)

In [36]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   polarity of tweet   1048572 non-null  int64 
 1   id of the tweet     1048572 non-null  int64 
 2   date of the tweet   1048572 non-null  object
 3   query               1048572 non-null  object
 4   user                1048572 non-null  object
 5   text of the tweet   1048572 non-null  object
dtypes: int64(2), object(4)
memory usage: 48.0+ MB


In [38]:
twitter_data.describe()

,polarity of tweet,id of the tweet
count,1.048572e+06,1.048572e+06
mean,9.482458e-01,1.976168e+09
std,1.701122e+00,2.300567e+08
min,0.000000e+00,1.467811e+09
25%,0.000000e+00,1.824526e+09
50%,0.000000e+00,1.990870e+09
75%,0.000000e+00,2.198903e+09
max,4.000000e+00,2.329206e+09


In [69]:
column_names = ['target', 'id', 'date', 'flag' , 'user', 'text']
twitter_data = pd.read_csv(path, names = column_names , encoding='ISO 8859-9')

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_10616\3572551955.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  twitter_data = pd.read_csv(path, names = column_names , encoding='ISO 8859-9')


In [71]:
twitter_data

target    object
id        object
date      object
flag      object
user      object
text      object
dtype: object

In [72]:
twitter_data.drop(index=twitter_data.index[0], axis =0 , inplace =True)

In [79]:
twitter_data['target'] = twitter_data['target'].astype(int)
twitter_data['id'] = twitter_data['id'].astype('longdouble')

In [80]:
twitter_data.head()

,target,id,date,flag,user,text
1,0,1.467811e+09,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1.467811e+09,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1.467811e+09,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [82]:
#counting the number of missing values in the dataset
twitter_data.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [81]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 1 to 1048572
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   target  1048572 non-null  int32  
 1   id      1048572 non-null  float64
 2   date    1048572 non-null  object 
 3   flag    1048572 non-null  object 
 4   user    1048572 non-null  object 
 5   text    1048572 non-null  object 
dtypes: float64(1), int32(1), object(4)
memory usage: 44.0+ MB


In [83]:
#checking the distribution of target column
twitter_data['target'].value_counts()

target
0    799996
4    248576
Name: count, dtype: int64

In [88]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [89]:
twitter_data['target'].value_counts()

target
0    799996
1    248576
Name: count, dtype: int64

0 --> negative tweet
1 --> Positive tweet

$Stemming$

process of reducing a word to its root word
ex : actor, actress , acting = act

In [90]:
port_stem = PorterStemmer()

In [92]:
def stemming(content) :
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [93]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

wait for atleast 50 minutes

In [96]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
1,0,1.467811e+09,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1.467811e+09,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1.467811e+09,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
5,0,1.467811e+09,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [97]:
print(twitter_data['stemmed_content'])

1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
5                                        kwesidei whole crew
                                 ...                        
1048568                              grandma make dinenr mum
1048569             mid morn snack time bowl chees noodl yum
1048570         shadela say like termini movi come like word
1048571                      destinyhop im great thaank wbuu
1048572                           cant wait til date weekend
Name: stemmed_content, Length: 1048572, dtype: object


In [98]:
print(twitter_data['target'])

1          0
2          0
3          0
4          0
5          0
          ..
1048568    1
1048569    1
1048570    1
1048571    1
1048572    1
Name: target, Length: 1048572, dtype: int32


In [104]:
#seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [105]:
print(X)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ...
 'shadela say like termini movi come like word'
 'destinyhop im great thaank wbuu' 'cant wait til date weekend']


In [106]:
print(Y)

[0 0 0 ... 1 1 1]


In [107]:
#spliting the data into training and test data
x_train ,x_test, y_train , y_test =  train_test_split(X,Y , test_size = 0.2 , stratify =Y , random_state=2)

In [108]:
print(X.shape,x_train.shape , x_test.shape)

(1048572,) (838857,) (209715,)


In [109]:
print(x_train)

['still headach' 'relax want start class tomorrow'
 'chatwitteddi awe nah mean got odd sens humor love' ...
 'wait till next time'
 'deadringa like differ place line hall amp front room also less like lightn like dog vomit'
 'mannnnn sooo piss dalworth carpet clean clean couch talkin chargin assum lol']


In [110]:
print(x_test)

['work right'
 'radiojayd head hope somewher without rain go head nashvil think get sick'
 'omg father day today complet forgot mean' ...
 'wonder make orchid bloom ever sinc got year ago'
 'tayytayi ah wish florida much fun' 'want dream come true']


In [111]:
#converting the textual data to numerical data

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [112]:
print(x_train)

  (0, 273528)	0.5829649440090278
  (0, 115224)	0.8124973071072612
  (1, 239489)	0.579063103740477
  (1, 310522)	0.3276500661640542
  (1, 271723)	0.4145495492134218
  (1, 53546)	0.4937476865142259
  (1, 293483)	0.3764216377122002
  (2, 48346)	0.5525271105523317
  (2, 20545)	0.33238117918437754
  (2, 201139)	0.315962476094208
  (2, 183538)	0.23317237793128717
  (2, 108150)	0.16845604955641608
  (2, 211771)	0.3401277625698359
  (2, 255156)	0.32278333598702397
  (2, 122574)	0.3906521972229178
  (2, 171849)	0.1707991305892916
  (3, 277520)	0.7695626526348762
  (3, 163787)	0.6385713144744077
  (4, 173108)	0.4200428879769875
  (4, 279292)	0.6960026437120873
  (4, 273921)	0.5823609638409303
  (5, 104766)	0.6407000068987219
  (5, 227792)	0.29607786809221026
  (5, 120778)	0.23035666317740128
  (5, 264074)	0.2389300173941699
  :	:
  (838855, 166641)	0.3815965222642205
  (838855, 10797)	0.14904366222659934
  (838855, 225790)	0.20520999820716782
  (838855, 9055)	0.1951272563477424
  (838855, 99462)

In [113]:
print(x_test)

  (0, 242146)	0.7990467141162862
  (0, 317367)	0.601268948690988
  (1, 104141)	0.1456474353259779
  (1, 106756)	0.13682705831668432
  (1, 115215)	0.4462325671446234
  (1, 120778)	0.18365828906528484
  (1, 202037)	0.3773977050490645
  (1, 234912)	0.554494936434281
  (1, 235353)	0.21328008447420474
  (1, 260830)	0.2093426490427187
  (1, 267135)	0.3099151244813293
  (1, 288988)	0.1762025280330974
  (1, 316055)	0.24679990448916767
  (2, 56476)	0.4511637704037005
  (2, 66881)	0.23957765671609776
  (2, 92106)	0.46805213763135145
  (2, 97305)	0.41387333488927097
  (2, 183538)	0.3766798545821832
  (2, 213917)	0.37103796023737806
  (2, 292744)	0.2629315063257491
  (3, 33660)	0.3594351359137039
  (3, 67260)	0.33581354302871425
  (3, 94368)	0.39570931253762687
  (3, 104814)	0.35380145846713007
  (3, 160164)	0.3266289945843121
  :	:
  (209711, 66881)	0.2068383879233664
  (209711, 108247)	0.33761748045776574
  (209711, 162235)	0.4411677333817011
  (209711, 170241)	0.24774156250918866
  (209711, 269

$Training the ML  Model$

LogisticRegression

In [114]:
model = LogisticRegression(max_iter = 1000)

In [115]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

$Model Evaluation$

Accuracy Score

In [116]:
#accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, x_train_prediction)


In [117]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.852310942151046


In [118]:
#accuracy score on the testing data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test, x_test_prediction)


In [119]:
print('Accuracy score on the testing data :', test_data_accuracy)

Accuracy score on the testing data : 0.8336075149607801


Model accuracy = 77.8%

model train accuracy = 81.1%

#saving the trained model

In [120]:

import pickle

In [121]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))


Using the Saved model for future predicitions

In [123]:
#loading the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))


In [124]:
x_new = x_test[200]
print(y_test[200])

prediction = model.predict(x_new)
print(prediction)

if (prediction[0] == 0):
    print('Negative Tweet')
else :
    print('Positive Tweet')

0
[0]
Negative Tweet


In [126]:
x_new = x_test[44]
print(y_test[44])

prediction = loaded_model.predict(x_new)
print(prediction)


if (prediction[0] == 0):
    print('Negative Tweet')
else :
    print('Positive Tweet')

1
[1]
Positive Tweet
